In [45]:
import torch
import json
import requests
from PIL import Image
from io import BytesIO
from torchvision import models, transforms

In [51]:
class ImagePrediction:
    
    def __init__(self):
        self.model =  models.inception_v3(pretrained=True)
        self.model.eval()
        self.load_class_labels()
        
    
    def load_class_labels(self):
        try:
            with open("imagenet_class_index.json") as f:
                class_idx = json.load(f)
        except FileNotFoundError:
            url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
            response = requests.get(url)
            class_names = response.json()
            class_idx = {str(i): [str(i), name] for i, name in enumerate(class_names)}
            with open("imagenet_class_index.json", "w") as f:
                json.dump(class_idx,f)
        
    def preprocess_image(self,image):
        preprocess = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            
        ])
        
        return preprocess(image)
    
    
                
    def predict_image(self,image_path,top_k=1):
        if image_path.startswith(("http://","https://")):
            response = requests.get(image_path)
            image = Image.open(BytesIO(response.content))
        else:
            image = Image.open(image_path)
            
        if image.mode != "RGB":
            image = image.convert("RGB")
            
        input_tensor = self.preprocess_image(image)
        input_batch = input_tensor.unsqueeze(0)
        
        with torch.no_grad():
            output = self.model(input_batch)
        
        
        return np.argmax(output)
    
    def result(self,image):
        prediction = image_predicator.predict_image(image)
        #print(f"Prediction: {prediction}")

        return prediction, data[str(prediction.item())][1]
#else:
    

In [ ]:
image_predicator = ImagePrediction()
print(image_predicator.result("https://nypost.com/wp-content/uploads/sites/2/2016/07/hot-dog-2.jpg?quality=90&strip=all&w=1280"))

d:\software\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\software\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(tensor(934), 'hot dog')


: 